## Introduction to Computational Graphs with PyTorch

by [Elvis Saravia](https://twitter.com/omarsar0)


In this notebook we provide a short introduction and overview of computational graphs using PyTorch.

There are several materials online that cover theory on the topic of computational graphs. However, I think it's much easier to learn the concept using code. I attempt to bridge the gap here which should be useful for beginner students.  

Inspired by Olah's article ["Calculus on Computational Graphs: Backpropagation"](https://colah.github.io/posts/2015-08-Backprop/), I've put together a few code snippets to get you started with computationsl graphs with PyTorch. This notebook should complement that article, so refer to it for more comprehensive explanations. In fact, I've tried to simplify the explanations and refer to them here.

### Why Computational Graphs?

When talking about neural networks in any context, [backpropagation](https://en.wikipedia.org/wiki/Backpropagation) is an important topic to understand because it is the algorithm used for training deep neural networks. 

Backpropagation is used to calculate derivatives which is what you need to keep optimizing parameters of the model and allowing the model to learn on the task at hand. 

Many of the deep learning frameworks today like PyTorch does the backpropagation out-of-the-box using [**automatic differentiation**](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html). 

To better understand how this is done it's important to talk about **computational graphs** which defines the flow of computations that are carried out throughout the network. Along the way we will use `torch.autograd` to demonstrate in code how this works.  

### Getting Started

Inspired by Olah's article on computational graphs, let's look at the following expression $e = (a + b) * (b+1)$. It helps to break it down to the following operations:

$$
\begin{aligned}&c=a+b \\&d=b+1 \\&e=c * d\end{aligned}
$$

This is not a neural network of any sort. We are just going through a very simple example of a chain of operations which you can be represented with computational graphs. 

Let's visualize these operations using a computational graph. Computational graphs contain **nodes** which can represent and input (tensor, matrix, vector, scalar) or **operation** that can be the input to another node. The nodes are connected by **edges**, which represent a function argument, they are pointers to nodes. Note that the computation graphs are directed and acyclic. The computational graph for our example looks as follows:

![](https://colah.github.io/posts/2015-08-Backprop/img/tree-def.png)

*Source: Christopher Olah (2015)*

We can evaluate the expression by setting our input variables as follows: $a=2$ and $b=1$. This will allow us to compute nodes up through the graph as shown in the computational graph above.  

Rather than doing this by hand, we can use the automatic differentation engine provided by PyTorch. 

Let's import PyTorch first:

In [1]:
import torch

Define the inputs like this:

In [2]:
a = torch.tensor([2.], requires_grad=True)
b = torch.tensor([1.], requires_grad=True)

Note that we used `requires_grad=True` to tell the autograd engine that every operation on them should be tracked. 

These are the operations in code:

In [3]:
c = a + b
d = b + 1
e = c * d

# grads populated for non-leaf nodes
c.retain_grad()
d.retain_grad()
e.retain_grad()

Note that we used `.retain_grad()` to allow gradients to be stored for non-leaf nodes as we are interested in inpecting those as well.

Now that we have our computational graph, we can check the result when evaluating the expression:

In [4]:
print(e)

tensor([6.], grad_fn=<MulBackward0>)


The output is a tensor with the value of `6.`, which verifies the results here: 

![](https://colah.github.io/posts/2015-08-Backprop/img/tree-eval.png)
*Source: Christopher Olah (2015)*

### Derivatives on Computational Graphs

Using the concept of computational graphs we are now interested in evaluating the **partial derivatives** of the edges of the graph. This will help in gathering the gradients of the graph. Remember that gradients are what we use to train the neural network and those calculations can be taken care of by the automatic differentation engine. 

The intuition is: we want to know, for example, if $a$ directly affects $c$, how does it affect it. In other words, if we change $a$ a little, how does $c$ change. This is referred to as the partial derivative of $c$ with respect to $a$.

You can work this by hand, but the easy way to do this with PyTorch is by calling `.backward()` on $e$ and let the engine figure out the values. The `.backward()` signals the autograd engine to calculate the gradients and store them in the respective tensors’ `.grad` attribute.

Let's do that now:

In [5]:
e.backward()

Now, let’s say we are interested in the derivative of $e$ with respect to $a$, how do we obtain this? In other words, we are looking for $\frac{\partial e}{\partial a}$.

Using PyTorch, we can do this by calling `a.grad`:

In [6]:
print(a.grad)

tensor([2.])


It is important to understand the intuition behind this. Olah puts it best:

>Let’s consider how $e$ is affected by $a$. If we change $a$ at a speed of 1, $c$ also changes at a speed of $1$. In turn, $c$ changing at a speed of $1$ causes $e$ to change at a speed of $2$. So $e$ changes at a rate of $1*2$ with respect to $a$.


In other words, by hand this would be:

$$
\frac{\partial e}{\partial \boldsymbol{a}}=\frac{\partial e}{\partial \boldsymbol{c}} \frac{\partial \boldsymbol{c}}{\partial \boldsymbol{a}} = 2 * 1
$$

You can verify that this is correct by checking the manual calculations by Olah. Since $a$ is not directly connectected to $e$, we can use some special rule which allows to sum over all paths from one node to the other in the computational graph and mulitplying the derivatives on each edge of the path together.

![](https://colah.github.io/posts/2015-08-Backprop/img/tree-eval-derivs.png)
*Source: Christopher Olah (2015)*

To check that this holds, let look at another example. How about caluclating the derivative of $e$ with respect to $b$, i.e., $\frac{\partial e}{\partial b}$?

We can get that through `b.grad`:

In [7]:
print(b.grad)

tensor([5.])


If you work it out by hand, you are basically doing the following:

$$
\frac{\partial e}{\partial b}=1 * 2+1 * 3
$$

It indicates how $b$ affects $e$ through $c$ and $d$. We are essentially summing over paths in the computational graph.

Here are all the gradients collected, including non-leaf nodes:

In [8]:
print(a.grad, b.grad, c.grad, d.grad, e.grad)

tensor([2.]) tensor([5.]) tensor([2.]) tensor([3.]) tensor([1.])


You can use the computational graph above to verify that everything is correct. This is the power of computational graphs and how they are used by automatic differentation engines. It's also a very useful concept to understand when developing neural networks architectures and their correctness.

### Next Steps

In this notebook, I've provided a simple and intuitive explanation to the concept of computational graphs using PyTorch. I highly recommend to go through [Olah's article](https://colah.github.io/posts/2015-08-Backprop/) for more on the topic.

In the next tutorial, I will be applying the concept of computational graphs to more advanced operations you typically see in a neural network. In fact, if you are interested in this, and you are feeling comfortable with the topic now, you can check out these two PyTorch tutorials:

- [A gentle introduction to `torch.autograd`](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html)
- [Automatic differentation with `torch.autograd`](https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html)

And here are some other useful references used to put this article together:

- [Hacker's guide to Neural Networks
](http://karpathy.github.io/neuralnets/)
- [Backpropagation calculus](https://www.youtube.com/watch?v=tIeHLnjs5U8&ab_channel=3Blue1Brown)

